## Magpie Toy Example

In [1]:
import transformers
import torch
import json
from transformers import AutoTokenizer
from fastchat.model import get_conversation_template

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda:0",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

#### Let's prepare the left-side template

In [3]:
with open("configs/model_configs.json", "r") as f:
    model_configs = json.load(f)
    model_config = model_configs[model_id]

# Prompt for extracting instructions from Llama-3-8B-Instruct
pre_query_template = model_config["pre_query_template"]
print(pre_query_template)

<|begin_of_text|><|start_header_id|>user<|end_header_id|>




#### Step 1: Extracting Instructions

In [4]:
#텍스트 생성이 종료될 조건을 정의
terminators = [
    tokenizer.eos_token_id, #128009
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),#128009
]

instruction = pipeline(
    pre_query_template,          # 입력 템플릿
    max_new_tokens=2048,         # 생성할 최대 토큰 수
    eos_token_id=terminators,    # 텍스트 생성을 종료할 토큰 ID
    do_sample=True,              # 샘플링을 사용하여 생성,동일한 입력에도 다양한 결과 생성가능
    temperature=1,               # 무작위성 조절 파라미터
    top_p=1,                      # 누적 확률 분포의 상위 p 비율
)

sanitized_instruction = instruction[0]['generated_text'][len(pre_query_template):].split("\n")[0]
print(f"Extracted Instruction: {sanitized_instruction}")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Extracted Instruction: I'm feeling a bit overwhelmed by all the different types of exercise and routines out there. Can you summarize the main types of exercise and routines for a beginner?


In [12]:
print(pre_query_template)
# instruction[0]['generated_text']

<|begin_of_text|><|start_header_id|>user<|end_header_id|>




#### Step 2: Generating Responses

In [13]:
# 대화 템플릿을 가져옵니다. 여기서는 "llama-3"이라는 이름의 템플릿을 사용합니다.
conv = get_conversation_template("llama-3")

# 현재 대화 템플릿을 출력하여 올바르게 설정되었는지 확인합니다.
print(conv)  # Make sure the conversation template is correct with name='llama-3' !!!

# 대화 템플릿에 사용자 역할(user)에 해당하는 메시지를 추가합니다.
# sanitized_instruction은 이전에 생성된 사용자 메시지입니다.
conv.append_message(conv.roles[0], sanitized_instruction)

# 대화 템플릿에 어시스턴트 역할(assistant)에 해당하는 메시지를 추가합니다.
# 현재는 응답을 생성하기 위한 자리 표시자로 None을 사용합니다.
conv.append_message(conv.roles[1], None)

# 현재 대화 상태를 기반으로 완성된 프롬프트를 생성합니다.
prompt = conv.get_prompt()

# 텍스트 생성 파이프라인을 호출하여 어시스턴트의 응답을 생성합니다.
response = pipeline(
    prompt,                # 생성할 텍스트의 입력 프롬프트
    max_new_tokens=2048,   # 생성할 최대 토큰 수
    eos_token_id=terminators,  # 텍스트 생성을 종료할 토큰 ID 리스트
    do_sample=True,        # 샘플링을 사용하여 텍스트 생성 여부
    temperature=1,         # 샘플링 시 무작위성 조절 파라미터
    top_p=1,               # 누적 확률 분포의 상위 p 비율
)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Conversation(name='one_shot', system_template='{system_message}', system_message="A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.", system_message_vision='', roles=('Human', 'Assistant'), messages=[['Human', 'Got any creative ideas for a 10 year old’s birthday?'], ['Assistant', "Of course! Here are some creative ideas for a 10-year-old's birthday party:\n1. Treasure Hunt: Organize a treasure hunt in your backyard or nearby park. Create clues and riddles for the kids to solve, leading them to hidden treasures and surprises.\n2. Science Party: Plan a science-themed party where kids can engage in fun and interactive experiments. You can set up different stations with activities like making slime, erupting volcanoes, or creating simple chemical reactions.\n3. Outdoor Movie Night: Set up a backyard movie night with a projector and a large screen or white sheet. Create a cozy seating 

### We have successfully created an instruction-response pair!

In [14]:
print(f"User's message: {sanitized_instruction}")
print(f"Assitant's response: {response[0]['generated_text'][len(prompt):]}")

User's message: I'm feeling a bit overwhelmed by all the different types of exercise and routines out there. Can you summarize the main types of exercise and routines for a beginner?
Assitant's response:  Don't worry! I'm here to help. As a beginner, it can be overwhelming to navigate the numerous types of exercises and routines. Here's a summary to get you started:

**1. Aerobic Exercise**: Also known as cardio, this type of exercise gets your heart rate up and improves cardiovascular health. Examples include:
	* Running, jogging, or walking
	* Swimming
	* Cycling
	* Dancing
	* Jumping rope

**2. Strength Training**: This type of exercise builds muscle and increases overall strength. You can focus on:
	* Free weights (dumbbells, barbells)
	* Resistance bands
	* Machines (at the gym)
	* Bodyweight exercises (push-ups, squats, lunges)

**3. Flexibility and Stretching**: This type of exercise improves flexibility and range of motion. Examples include:
	* Yoga
	* Pilates
	* Tai chi or oth